In [1]:
import pandas as pd
d = pd.read_csv("Youtube01-Psy.csv")

In [3]:
d.tail()
len(d)

350

In [4]:
len(d.query('CLASS == 1'))

175

In [5]:
len(d.query('CLASS == 0'))

175

In [6]:
len(d)

350

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [15]:
dvec = vectorizer.fit_transform(d['CONTENT'])

In [10]:
dvec

<350x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 4354 stored elements in Compressed Sparse Row format>

In [14]:
analyze = vectorizer.build_analyzer()

In [12]:
print(d['CONTENT'][349])
analyze(d['CONTENT'][349])

The first billion viewed this because they thought it was really cool, the  other billion and a half came to see how stupid the first billion were...﻿


['the',
 'first',
 'billion',
 'viewed',
 'this',
 'because',
 'they',
 'thought',
 'it',
 'was',
 'really',
 'cool',
 'the',
 'other',
 'billion',
 'and',
 'half',
 'came',
 'to',
 'see',
 'how',
 'stupid',
 'the',
 'first',
 'billion',
 'were']

In [19]:
vectorizer.get_feature_names_out()

array(['00', '000', '02', ..., 'ｆａｎｃy', 'ｉｓ', 'ｔｈｉｓ'], dtype=object)

In [20]:
dshuf = d.sample(frac=1)

In [21]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train['CONTENT']) # fit bag-of-words on training set
d_test_att = vectorizer.transform(d_test['CONTENT']) # reuse on testing set
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']

In [22]:
d_train_att

<300x1269 sparse matrix of type '<class 'numpy.int64'>'
	with 3625 stored elements in Compressed Sparse Row format>

In [23]:
d_test_att

<50x1269 sparse matrix of type '<class 'numpy.int64'>'
	with 576 stored elements in Compressed Sparse Row format>

In [24]:
#Initialiye the random forrest classifier with 80 decision trees

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

In [25]:
#train the model
clf.fit(d_train_att, d_train_label)

RandomForestClassifier(n_estimators=80)

In [26]:
#Check how well it performs
clf.score(d_test_att, d_test_label)

0.94

In [27]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

array([[20,  1],
       [ 2, 27]], dtype=int64)

In [28]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
# show average score and +/- two standard deviations away (covering 95% of scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.02)


In [30]:
# load all datasets and combine them
d = pd.concat([pd.read_csv("Youtube01-Psy.csv"),
               pd.read_csv("Youtube02-KatyPerry.csv"),
               pd.read_csv("Youtube03-LMFAO.csv"),
               pd.read_csv("Youtube04-Eminem.csv"),
               pd.read_csv("Youtube05-Shakira.csv")])

In [31]:
len(d)

1956

In [32]:
len(d.query('CLASS == 1'))

1005

In [33]:
len(d.query('CLASS == 0'))

951

In [34]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [37]:
# set up a pipeline
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random forest', RandomForestClassifier()),
])
pipeline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [38]:
#This is the training set of the data 1500 
pipeline.fit(d_content[:1500],d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [39]:
pipeline.score(d_content[1500:], d_label[1500:])

0.9451754385964912

In [40]:
pipeline.predict(["what a neat video!"])

array([0], dtype=int64)

In [43]:
pipeline.predict(["please buy my shit"])

array([1], dtype=int64)

In [44]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.02)


In [45]:
# add tfidf
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(),
                          TfidfTransformer(norm=None),
                          RandomForestClassifier())

In [46]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.02)


In [47]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

In [51]:
# parameter search
parameters = {
    'countvectorizer__max_features': (None, 1000, 2000),
    'countvectorizer__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'countvectorizer__stop_words': ('english', None),
    'tfidftransformer__use_idf': (True, False), # effectively turn on/off tfidf
    'randomforestclassifier__n_estimators': (20, 50, 100,150)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

In [52]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100,
                                                                  150),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [50]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.961
Best parameters set:
	countvectorizer__max_features: None
	countvectorizer__ngram_range: (1, 1)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 100
	tfidftransformer__use_idf: False
